to access the openAI API, you need an API key which you can get it from  https://platform.openai.com/api-keys
once you have a key then you can either save it as an environment variable or hard code in as needed from simplicity

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv(override=True)
#os.environ["OPENAI_API_KEY"] = "your key"



True

In [2]:
import os

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"], model_name="gpt-4-0125-preview")
output_parser = StrOutputParser()


Use document loaders to load data from a source as Document

Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [ ]:
%pip install pypdf

you can also load directory using directory loader.
and split a long document into smaller chunks that can fit into your model's context window.

In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter


raw_documents = DirectoryLoader('../article', glob="*.csv", loader_cls=CSVLoader).load()
#len(raw_documents)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)





Facebook AI Similarity Search (Faiss) is a library for efficient similarity search and clustering of dense vectors.

search result will be fetched by retriever and it'll be used as context

In [15]:
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings


vectorstore = FAISS.from_documents(documents, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]))

retriever = vectorstore.as_retriever()


invoke question

In [22]:
from pprint import pprint

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt 
    | model 
    | output_parser
)

pprint(chain.invoke("tell me about funnnnyIO"))

('Emerging FunnyIO technologies are at the forefront of revolutionizing data '
 'transfer and processing across various computing environments. Among these '
 'advanced connectivity standards, notable mentions include USB4, Thunderbolt '
 '4, PCIe 5.0, and FunnyIO 6.0. These technologies are critical for meeting '
 'the increasing demands of modern computing by offering high-speed I/O '
 'capabilities.\n'
 '\n'
 'USB4, in particular, is highlighted as a highly anticipated development in '
 'the realm of high-speed I/O technologies. It represents the latest iteration '
 'of the Universal Serial Bus standard and boasts substantial improvements in '
 'data transfer speeds, with capabilities reaching up to 40 Gbps over a single '
 'cable. This enhancement is significant for a variety of applications, '
 'including connecting external storage devices, displays, and other '
 'peripherals that demand high bandwidth. For users dealing with large volumes '
 'of data, USB4 facilitates faster bac